In [1]:
import os
%pwd

'/home/naseem/RandD/Deeplearning-MLOps/research'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [6]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from url
        """
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            root_dir = self.config.root_dir
            os.makedirs(root_dir, exist_ok=True)
            
            logger.info(f"Downloading data from {dataset_url} -> {zip_download_dir}")
            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} -> {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_path)

In [9]:
%cd ../

/home/naseem/RandD/Deeplearning-MLOps


/home/naseem/anaconda3/envs/mlops/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
%pwd

'/home/naseem/RandD/Deeplearning-MLOps'

In [19]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-15 15:05:42,296: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-15 15:05:42,297: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-15 15:05:42,298: INFO: common: created directory: artifacts]
[2025-04-15 15:05:42,298: INFO: common: created directory: artifacts/data_ingestion]
[2025-04-15 15:05:42,298: INFO: 3986763959: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing -> artifacts/data_ingestion/data.zip]


ConnectionError: HTTPSConnectionPool(host='drive.google.com', port=443): Max retries exceeded with url: /uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f71bd767130>: Failed to resolve 'drive.google.com' ([Errno -3] Temporary failure in name resolution)"))